In [4]:
import threading

In [5]:
threading.active_count()

8

In [6]:
threading.enumerate()

[<_MainThread(MainThread, started 4702164480)>,
 <Thread(IOPub, started daemon 123145453342720)>,
 <Heartbeat(Heartbeat, started daemon 123145470132224)>,
 <Thread(Thread-2 (_watch_pipe_fd), started daemon 123145487994880)>,
 <Thread(Thread-3 (_watch_pipe_fd), started daemon 123145504784384)>,
 <ControlThread(Control, started daemon 123145521573888)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145538363392)>,
 <ParentPollerUnix(Thread-1, started daemon 123145555689472)>]

In [7]:
threading.active_count(), threading.activeCount()

/var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_31000/2258099811.py:1: DeprecationWarning: activeCount() is deprecated, use active_count() instead
  threading.active_count(), threading.activeCount()


(8, 8)

In [9]:
main = threading.current_thread()

In [10]:
main

<_MainThread(MainThread, started 4702164480)>

In [11]:
main.name, main.ident, main.native_id, main.daemon, main.is_alive()

('MainThread', 4702164480, 3325268, False, True)

In [12]:
threading.get_ident(), threading.get_native_id()

(4702164480, 3325268)

In [13]:
threading.get_ident?

Docstring:
get_ident() -> integer

Return a non-zero integer that uniquely identifies the current thread
amongst other threads that exist simultaneously.
This may be used to identify per-thread resources.
Even though on some platforms threads identities may appear to be
allocated consecutive numbers starting at 1, this behavior should not
be relied upon, and the number should be seen purely as a magic cookie.
A thread's identity may be reused for another thread after it exits.
Type:      builtin_function_or_method

In [15]:
threading.enumerate()[3].ident

123145487994880

In [16]:
import time

In [26]:
class DataProcessor(threading.Thread):
    def __init__(self, n):
        super().__init__()
        self.n = n

    def run(self):
        print(f"{self.name=}, {self.ident=}, {self.native_id=}, {self.n=}")

        for i in range(self.n):
            print(f"{i=}")
            time.sleep(1)

data_th = DataProcessor(10)
print(f"{data_th.is_alive()=}")
data_th.start()

data_th.is_alive()=False
self.name='Thread-8', self.ident=123145572478976, self.native_id=3411771, self.n=10
i=0
i=1
i=2
i=3


In [19]:
print(121)

121
i=8
i=9


In [27]:
data_th.is_alive()

True

i=4
i=5
i=6
i=7
i=8
i=9


In [28]:
data_th.is_alive()

False

In [33]:
class DataProcessor(threading.Thread):
    def __init__(self, n):
        super().__init__()
        self.n = n

    def run(self):
        print(f"{self.name=}, {self.ident=}, {self.native_id=}, {self.n=}")

        for i in range(self.n):
            print(f"{i=}")
            time.sleep(1)

data_th = DataProcessor(10)
print(f"{data_th.is_alive()=}")
data_th.start()
data_th.join(timeout=5)
print("after join")

data_th.is_alive()=False
self.name='Thread-12', self.ident=123145572478976, self.native_id=3415056, self.n=10
i=0
i=1
i=2
i=3
i=4
after join
i=5
i=6
i=7
i=8


In [39]:
N = 10 ** 8

def counter(a, b):
    while a < b:
        a += 1

In [40]:
%%time

counter(0, N)

CPU times: user 5.06 s, sys: 90.6 ms, total: 5.15 s
Wall time: 6.89 s


In [43]:
%%time

count_th = threading.Thread(target=counter, args=(0, N))
count_th.start()
count_th.join()

CPU times: user 5.23 s, sys: 125 ms, total: 5.35 s
Wall time: 7.33 s


In [47]:
%%time

N = 10 ** 8
N_THREADS = 10
WORK_SIZE = N / N_THREADS

def counter(a, b):
    print(f"{threading.current_thread().name}")
    while a < b:
        a += 1
# 0 - 100
# 100 - 200
# ... - N

threads = [
    threading.Thread(
        target=counter,
        args=(i * WORK_SIZE, (i + 1) * WORK_SIZE),
        name=f"counter_th_{i}",
    )
    for i in range(N_THREADS)
]
for th in threads:
    th.start()

for th in threads:
    th.join()

counter_th_0
counter_th_1
counter_th_2
counter_th_3
counter_th_4
counter_th_5
counter_th_6
counter_th_7
counter_th_8
counter_th_9
CPU times: user 5.39 s, sys: 227 ms, total: 5.62 s
Wall time: 7.3 s


In [48]:
import sys

In [49]:
sys.getswitchinterval()

0.005

In [50]:
sys.setswitchinterval

<function sys.setswitchinterval(interval, /)>

In [51]:
%%time

N = 10 ** 6
N_THREADS = 10
WORK_SIZE = N / N_THREADS

count = 0


def counter(a, b):
    print(f"{threading.current_thread().name}")
    global count
    
    while a < b:
        tmp = count
        a += 1
        tmp += 1
        lst = list(range(-10, 10))
        y = sorted(lst, key=lambda x: x ** 2)
        count = tmp


threads = [
    threading.Thread(
        target=counter,
        args=(i * WORK_SIZE, (i + 1) * WORK_SIZE),
        name=f"counter_th_{i}",
    )
    for i in range(N_THREADS)
]

print(f"{count=}")

for th in threads:
    th.start()

for th in threads:
    th.join()

print(f"{count=}")

count=0
counter_th_0
counter_th_1
counter_th_2
counter_th_3
counter_th_4
counter_th_5
counter_th_6
counter_th_7
counter_th_8
counter_th_9
count=138418
CPU times: user 3.82 s, sys: 181 ms, total: 4 s
Wall time: 5.35 s


In [52]:

%%time

N = 10 ** 6
N_THREADS = 10
WORK_SIZE = N / N_THREADS

count = 0
lock = threading.Lock()


def counter(a, b):
    print(f"{threading.current_thread().name}")
    global count
    
    while a < b:
        lock.acquire()
        tmp = count
        a += 1
        tmp += 1
        lst = list(range(-10, 10))
        y = sorted(lst, key=lambda x: x ** 2)
        count = tmp
        lock.release()


threads = [
    threading.Thread(
        target=counter,
        args=(i * WORK_SIZE, (i + 1) * WORK_SIZE),
        name=f"counter_th_{i}",
    )
    for i in range(N_THREADS)
]

print(f"{count=}")

for th in threads:
    th.start()

for th in threads:
    th.join()

print(f"{count=}")

count=0
counter_th_0
counter_th_1
counter_th_2
counter_th_3
counter_th_4
counter_th_5
counter_th_6
counter_th_7
counter_th_8
counter_th_9
count=1000000
CPU times: user 3.39 s, sys: 200 ms, total: 3.59 s
Wall time: 4.56 s


In [53]:
from urllib.request import urlopen

In [54]:
def fetch_url(url):
    resp = urlopen(url)
    return resp

In [55]:
URL = "http://ru.wikipedia.org/wiki/Python"

In [56]:
resp = fetch_url(URL)

In [60]:
%%time

URL = "http://ru.wikipedia.org/wiki/Python"
URLS = [URL] * 20


def fetch_url(url):
    resp = urlopen(url)
    return resp


def fetch_batch_urls(urls):
    for url in urls:
        fetch_url(url)


fetch_batch_urls(URLS)

CPU times: user 88.4 ms, sys: 29.6 ms, total: 118 ms
Wall time: 5.92 s


In [68]:
%%time

URL = "http://ru.wikipedia.org/wiki/Python"
URLS = [URL] * 20
N_THREADS = 10
WORK_SIZE = len(URLS) // N_THREADS


def fetch_url(url, sem):
    with sem:
        resp = urlopen(url)
    return resp


def fetch_batch_urls(urls, sem):
    print(f"{threading.current_thread().name}")
    t1 = time.time()
    for url in urls:
        fetch_url(url, sem)
    t2 = time.time()
    print(f"{threading.current_thread().name}, {t2 - t1=}")


def run():
    sem = threading.Semaphore(2)
    
    threads = [
        threading.Thread(
            target=fetch_batch_urls,
            args=(URLS[i * WORK_SIZE : (i + 1) * WORK_SIZE], sem),
            name=f"fetcher_th_{i}",
        )
        for i in range(N_THREADS)
    ]
    
    for th in threads:
        th.start()
    
    for th in threads:
        th.join()

run()

fetcher_th_0
fetcher_th_1
fetcher_th_2
fetcher_th_3
fetcher_th_4
fetcher_th_5
fetcher_th_6
fetcher_th_7
fetcher_th_8
fetcher_th_9
fetcher_th_0, t2 - t1=1.6411490440368652
fetcher_th_1, t2 - t1=1.6497299671173096
fetcher_th_3, t2 - t1=1.9293251037597656
fetcher_th_2, t2 - t1=1.938978910446167
fetcher_th_4, t2 - t1=2.1756319999694824
fetcher_th_5, t2 - t1=2.1962778568267822
fetcher_th_6, t2 - t1=2.43249773979187
fetcher_th_7, t2 - t1=2.460860013961792
fetcher_th_8, t2 - t1=2.702885866165161
fetcher_th_9, t2 - t1=2.754934072494507
CPU times: user 113 ms, sys: 41.8 ms, total: 155 ms
Wall time: 2.76 s


In [69]:
import queue

In [72]:
%%time

URL = "http://ru.wikipedia.org/wiki/Python"
URLS = [URL] * 100
N_THREADS = 10
WORK_SIZE = len(URLS) // N_THREADS


def fetch_url(url, sem):
    with sem:
        resp = urlopen(url)
    return resp


def worker_fetch(que, sem):
    print(f"{threading.current_thread().name}")
    t1 = time.time()
    count = 0
    while True:
        url = que.get()
        if url is None:
            que.put(url)
            break

        count += 1
        fetch_url(url, sem)

    t2 = time.time()
    print(f"{threading.current_thread().name}, {t2 - t1=}, {count=}")


def run():
    sem = threading.Semaphore(2)
    que = queue.Queue()
    for url in URLS:
        que.put(url)
    que.put(None)
    
    threads = [
        threading.Thread(
            target=worker_fetch,
            args=(que, sem),
            name=f"fetcher_th_{i}",
        )
        for i in range(N_THREADS)
    ]
    
    for th in threads:
        th.start()
    
    for th in threads:
        th.join()

run()

fetcher_th_0fetcher_th_1
fetcher_th_2

fetcher_th_3
fetcher_th_4
fetcher_th_5
fetcher_th_6
fetcher_th_7
fetcher_th_8
fetcher_th_9
fetcher_th_8, t2 - t1=12.704658031463623, count=10
fetcher_th_4, t2 - t1=13.310893774032593, count=9
fetcher_th_3, t2 - t1=13.319364786148071, count=9
fetcher_th_2, t2 - t1=13.552070140838623, count=12
fetcher_th_1, t2 - t1=13.57404112815857, count=11
fetcher_th_7, t2 - t1=13.817423105239868, count=9
fetcher_th_6, t2 - t1=13.939372777938843, count=10
fetcher_th_9, t2 - t1=14.133655071258545, count=9
fetcher_th_0, t2 - t1=14.198372840881348, count=9
fetcher_th_5, t2 - t1=14.395900249481201, count=12
CPU times: user 492 ms, sys: 161 ms, total: 653 ms
Wall time: 14.4 s


In [74]:
def bad_data_process():
    print("start bad")
    raise Exception("BAD_ERROR")


th = threading.Thread(target=bad_data_process)
th.start()
th.join()
print("finish bad")

Exception in thread Thread-18 (bad_data_process):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/Users/g.kandaurov/venv_education/vk_py312/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_31000/2875439389.py", line 3, in bad_data_process
Exception: BAD_ERROR


start bad
finish bad


In [75]:
def process_error(*args, **kwargs):
    print(f"process_error {args=}, {kwargs=}")

threading.excepthook = process_error


def bad_data_process():
    print("start bad")
    raise Exception("BAD_ERROR")


th = threading.Thread(target=bad_data_process)
th.start()
th.join()
print("finish bad")

start bad
process_error args=(_thread._ExceptHookArgs(exc_type=<class 'Exception'>, exc_value=Exception('BAD_ERROR'), exc_traceback=<traceback object at 0x10e342540>, thread=<Thread(Thread-19 (bad_data_process), started 123145590341632)>),), kwargs={}
finish bad
